# MBAM model reduction for the MWC model of BK
Model taken from Aldrich 2002 (Equation 1) <br>
Initial parameters taken from Miranda 2013 PNAS, Suppl fig 5 <br>
Algorithm as in Transtrum 2015 (Equation 49-51) - except for signs

The code below iteratively finds limits (boundaries) and stiff nonlinear combinations of sloppy parameters for the MWC model of BK, while preserving the model output function $P_{open}(V, [Ca] \ | \ \theta)$.

The iterative model reduction proceeds as follows:
+ $\theta = \{L_0, z_L, J_0, z_J, K_d, C, D, E \}$
    * $z_L -> 0$
    
    
+ $\phi^1 = \{L_0, J_0, z_J, K_d, C, D, E \}$
    * $L_0^{1/4} \rightarrow 0, \quad C \rightarrow \infty, \quad D \rightarrow \infty, \quad E \rightarrow 0 $
    

+ $\phi^2 = \{J_0, z_J, K_d, L_0^{1/4}C, L_0^{1/4}D, L_0^{-1/4}E \}$ 
    * $L_0^{1/4}C \rightarrow 0, \quad L_0^{-1/4}E \rightarrow \infty $
    
    
+ $\phi^3 = \{J_0, z_J, K_d, CE, L_0^{1/4}D \}$ 
    * $J_0 \rightarrow 0, \quad L_0^{1/4}D \rightarrow \infty $
 
 
+ $\phi^4 = \{J_0L_0^{1/4}D, z_J, K_d, CE  \}$ 
    * $K_d \rightarrow \infty, \quad CE \rightarrow \infty $
 
 
+ $\phi^5 = \{J_0L_0^{1/4}D, z_J, CEK_d^{-1}  \}$ 

The $P_{open}(V, [Ca] \ | \ \phi^5)$ function is an excellent approximation of the original (way below experimental error), despite having only 3 parameters. Each of the parameters are meaningful, the first one describing the slope of the voltage sensitivity, the second the middle point of the voltage sensitivity and the third the calcium sensitivity.
Note that the 4 parameter-model ($\phi^4$) describes both the voltage and the calcium sensitivity curves as two-parameter sigmoids. The allosteric MWC model introduces nonlinear interaction between these two sensitivity curves.

In [2]:
using NBInclude #Includes for ipynb-s
#using JLD # Julia data I/O
nbinclude("MBAM.ipynb")
nbinclude("BK_functions/BK_functions.ipynb")


BK_cost (generic function with 1 method)

In [3]:
# Create grid to evaluate over
#=
using Iterators
Ca=([0.0, 0.7, 4.0, 12.0, 22.0, 55.0, 70.0, 95.0]*1e-6)
V=collect(-100:25:200)*1e-3
Ca = float(Ca)
V = float(V)

x_grid = Array(Any, (length(Ca)*length(V),))
iterind = 0;
for j1 in product(Ca, V)
    iterind += 1;
    x_grid[iterind] = collect(j1)
end
size(x_grid[1])
=#

In [ ]:
#=
# Single MBAM call that runs through all iterations of BK model reduction with NON-AUTOMATED model reduction
phi0 = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0];
outp = MBAM(BK_simulator, phi0, x_grid, model_iters=[10, 11, 12, 13, 14], 
move_dir=[-1, -1, 1, 1, -1], boundary_time=10, verbose=1, reduce_func=BK_reduce);
=#


  0.716084 seconds (665.10 k allocations: 31.533 MB, 0.78% gc time)
Model iteration 10, Single g_cost call takes: 
  0.060158 seconds (76.52 k allocations: 3.465 MB)
Model iteration 10, initial values:
      D = [  2.45e-05,   2.83e-04,   1.32e-03,   4.91e-02,   8.41e-02,   3.34e-01,   5.48e+00,   2.81e+01]

# Fit the reduced models to data
Using data from Aldrich 2010 Fig 4c,d fit simplified models

In [430]:
# Read the data
data_4c_control = readcsv("Figures/aldrich_fig4c_control.csv")[1:end-1,:]
data_4c_lrrc26 = readcsv("Figures/aldrich_fig4c_lrrc26.csv")
data_4d_control = readcsv("Figures/aldrich_fig4d_control.csv")
data_4d_lrrc26 = readcsv("Figures/aldrich_fig4d_lrrc26.csv");


# Models for simultaneously fitting all 4 curves

In [388]:
# join the data
data_x = [data_4c_control[:,1]; data_4c_lrrc26[:,1]; data_4d_control[:,1]; data_4d_lrrc26[:,1]]*1e-3;
data_y = [data_4c_control[:,2]; data_4c_lrrc26[:,2]; log10(data_4d_control[:,2]); log10(data_4d_lrrc26[:,2])];
data_in_log = [zeros(length(data_4c_control[:,1]),1); zeros(length(data_4c_lrrc26[:,1]),1); 
    ones(length(data_4d_control[:,1]),1); ones(length(data_4d_lrrc26[:,1]),1)].==1;
dataset_ranges = Any[1:length(data_4c_control[:,1]),
    (length(data_4c_control[:,1])+1):(length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])),
    (length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])+1):(length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])+length(data_4d_control[:,1])),
    (length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])+length(data_4d_control[:,1])+1):length(data_x)]

# Tradeoff between log-space fit and original space fit by norm of data there
w_log = norm([data_4c_control[:,2]; data_4c_lrrc26[:,2]])./norm([data_4d_control[:,2]; data_4d_lrrc26[:,2]])

0.9766267267700984

In [395]:
# Return results in log
function to_log(x::AbstractArray, in_log::AbstractArray)
    x = copy(x);
    x[x.<1e-12]=1e-12
    x[find(a->a==true, in_log)] = log10(x[find(a->a==true, in_log)])
    return x
end

function scale_data(x::AbstractArray, to_scale::AbstractArray, w)
    x = copy(x);
    x[find(a->a==true, to_scale)] *= w;
    return x
end

scale_data (generic function with 1 method)

## Models

In [739]:
using LsqFit
using Physical

elec_resp = (Volt*(-1)*e_electron/(k_boltzmann*304*Kelvin));


function model_orig(x::AbstractArray, ϕ::AbstractArray)
    D = ϕ[1]; L0 = ϕ[2]; J0 = ϕ[3]; zJ = ϕ[4]; zL = ϕ[5];
    Popen = (1+
    (1.+J0.*exp(-zJ .* x .* elec_resp )).^4
    ./
    ( L0.*exp(-zL .* x .* elec_resp ).*(1+D.*J0.*exp(-zJ .* x .* elec_resp )).^4)
    ).^-1
    
    logPopen = copy(Popen);
    for p1 = 1:length(logPopen)
        logPopen[p1] = Popen[p1]>0?log(Popen[p1]):-6
    end
    return Popen
end

function model_red(x::AbstractArray, ϕ::AbstractArray)
    DL0 = ϕ[1]; J0 = ϕ[2]; zJ = ϕ[3]; zL = ϕ[4];
    Popen = (1+
    (1.+J0.*exp(-zJ .* x .* elec_resp )).^4
    ./
    (exp(-zL .* x .* elec_resp ).*(DL0.*J0.*exp(-zJ .* x .* elec_resp )).^4)
    ).^-1
    return Popen
end

function model_Dmoves(x::AbstractArray, ϕ::AbstractArray; in_log=(zeros(size(x)).==1), data_ranges=Any[collect(1:length(x))], w_log=1)
    y_out = zeros(size(x))
    y_out[data_ranges[1]] = model_orig(x[data_ranges[1]], [ϕ[1]; ϕ[2:5]])
    y_out[data_ranges[2]] = model_orig(x[data_ranges[2]], [ϕ[6]; ϕ[2:5]])
    y_out[data_ranges[3]] = model_orig(x[data_ranges[3]], [ϕ[1]; ϕ[2:5]])
    y_out[data_ranges[4]] = model_orig(x[data_ranges[4]], [ϕ[6]; ϕ[2:5]])
    return scale_data(to_log(y_out, in_log), in_log, w_log);
end


function model_LDmoves(x::AbstractArray, ϕ::AbstractArray; in_log=(zeros(size(x)).==1), data_ranges=Any[collect(1:length(x))], w_log=1)
    y_out = zeros(size(x))
    y_out[data_ranges[1]] = model_red(x[data_ranges[1]], [ϕ[1]; ϕ[2:4]])
    y_out[data_ranges[2]] = model_red(x[data_ranges[2]], [ϕ[5]; ϕ[2:4]])
    y_out[data_ranges[3]] = model_red(x[data_ranges[3]], [ϕ[1]; ϕ[2:4]])
    y_out[data_ranges[4]] = model_red(x[data_ranges[4]], [ϕ[5]; ϕ[2:4]])
    return scale_data(to_log(y_out, in_log), in_log, w_log);
end


function model_red_allmoves(x::AbstractArray, ϕ::AbstractArray; in_log=(zeros(size(x)).==1), data_ranges=Any[collect(1:length(x))], w_log=1)
    y_out = zeros(size(x))
    y_out[data_ranges[1]] = model_red(x[data_ranges[1]], [ϕ[1]; ϕ[2:4]])
    y_out[data_ranges[2]] = model_red(x[data_ranges[2]], [ϕ[5]; ϕ[6:8]])
    y_out[data_ranges[3]] = model_red(x[data_ranges[3]], [ϕ[1]; ϕ[2:4]])
    y_out[data_ranges[4]] = model_red(x[data_ranges[4]], [ϕ[5]; ϕ[6:8]])
    return scale_data(to_log(y_out, in_log), in_log, w_log);
end

model_red_allmoves (generic function with 1 method)

In [742]:
weight_of_log_fits = 0.53;

phi0 = [21.0, 3.6e-6, 0.048, 0.59, 0.29]

fit_orig = LsqFit.curve_fit((a,b)->model_Dmoves(a,b, in_log=data_in_log, data_ranges=dataset_ranges, w_log=weight_of_log_fits), 
                                data_x, scale_data(data_y, data_in_log, weight_of_log_fits), [phi0; 412]);

orig_cost=sum(scale_data(fit_orig.resid, data_in_log, weight_of_log_fits).^2)

par_results = zeros(5,100)
par_costs = zeros(1,100)
for i1 = 1:100
    fit_red = LsqFit.curve_fit((a,b)->model_LDmoves(a,b, in_log=data_in_log, data_ranges=dataset_ranges, w_log=weight_of_log_fits), 
    data_x, scale_data(data_y, data_in_log, weight_of_log_fits), [0.91; phi0[3:5]; 3]+randn(5,));
    
    par_results[:,i1] = fit_red.param
    par_costs[i1] = sum(scale_data(fit_red.resid, data_in_log, weight_of_log_fits).^2)
end

2.7947468214805813 (predicted_residual) >
2.79474682148058 (residual) + 4.440892098500626e-16 (eps)


In [743]:
@show par_costs
@show minimum(par_costs)
@show orig_cost

par_costs = [0.952624003325932 0.952624001798078 0.952758440869936 0.9526240019645689 0.9550494078579835 0.9526240033359973 1.2045370620422367 0.95262400335743 4.631348983795834 0.9526240034109648 1.2045370656416028 1.2045370747565811 4.631353642383346 4.631332407612087 1.2045370635315749 0.9526240033713835 0.9530624651522086 1.2045370620917648 0.952624001859866 1.2045370608064059 0.952624003326169 4.631339841365774 1.204537062602952 4.631359393703677 4.631316988188574 16.461625347820313 1.2045370810210825 0.9526240033307831 0.9531558373052689 1.1950614774282151 0.9539502856818991 0.9526240033484182 1.2045370775514903 1.204537061797855 4.631343464911464 16.46030867831722 4.631361294943664 1.2045370629922871 1.2623436360875349 4.631423698132135 0.9545258047216065 1.2045370905233486 1.2045376148604214 0.9526240020489718 0.9572401864075535 1.2045370622019833 0.9569682533334134 1.2045370626252416 0.952624003379387 0.9537369436938108 0.9521809529558161 17.536575980837682 0.9526240033610036 

0.059592538702537244

In [744]:
y_pred_orig = model_Dmoves(data_x, [phi0, 412], data_ranges=dataset_ranges, in_log=data_in_log);
y_pred_origfit = model_Dmoves(data_x, fit_orig.param, data_ranges=dataset_ranges, in_log=data_in_log);
y_pred_redfit = model_LDmoves(data_x, par_results[:,sortperm(par_costs[:])[1]], data_ranges=dataset_ranges, in_log=data_in_log);

In [747]:
using PlotlyJS


p1 = Plot([scatter(;x=data_x[dataset_ranges[3]], y=data_y[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=data_y[dataset_ranges[4]]),
    scatter(;x=data_x[dataset_ranges[3]], y=y_pred_origfit[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=y_pred_origfit[dataset_ranges[4]]),
    scatter(;x=data_x[dataset_ranges[3]], y=y_pred_redfit[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=y_pred_redfit[dataset_ranges[4]])
    ]
        )

p2 = Plot([scatter(;x=data_x[dataset_ranges[1]], y=data_y[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=data_y[dataset_ranges[2]]),
    scatter(;x=data_x[dataset_ranges[1]], y=y_pred_origfit[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=y_pred_origfit[dataset_ranges[2]]),
    scatter(;x=data_x[dataset_ranges[1]], y=y_pred_redfit[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=y_pred_redfit[dataset_ranges[2]])
    ]
        )

plt = plot([p1
    p2])


In [748]:
PlotlyJS.relayout!(plt, height=1000)

In [645]:
#@show fit_orig.param
#@show fit_red.param

fit_orig.param = [27.926981569820867,4.342907342760481e-6,0.07138997056602382,0.45444758291228826,0.20245710174748585,283.17427917720755]
fit_red.param = [0.46507575681332425,0.1372273359579971,0.6006739127824584,0.5849606971604974,1.6326389308283835,1.2502595802033498,0.30585955260437236,1.271562666687221]


8-element Array{Float64,1}:
 0.465076
 0.137227
 0.600674
 0.584961
 1.63264 
 1.25026 
 0.30586 
 1.27156 

# Model reduction while fitting log and classic data simultaneously

In [691]:
# Enable all parameters to move between Control and LRRC26, see what gets reduced

function model_allmoves(x::AbstractArray, ϕ::AbstractArray; in_log=(zeros(size(x)).==1), data_ranges=Any[collect(1:length(x))], w_log=1)
    y_out1 = model_orig(x[data_ranges[1]], [ϕ[1]; ϕ[2:5]])
    y_out2 = model_orig(x[data_ranges[2]], [ϕ[6]; ϕ[7:10]])
    y_out3 = model_orig(x[data_ranges[3]], [ϕ[1]; ϕ[2:5]])
    y_out4 = model_orig(x[data_ranges[4]], [ϕ[6]; ϕ[7:10]])
    y_out = [y_out1; y_out2; y_out3; y_out4]
    return scale_data(to_log(y_out, in_log), in_log, w_log);
end

model_allmoves (generic function with 1 method)

In [727]:
# Find best fit point
weight_of_log_fits = w_log;

par_results = zeros(10,100)
par_costs = zeros(1,100)
for i1 = 1:100
    fit_orig = LsqFit.curve_fit((a,b)->model_allmoves(a,b, in_log=data_in_log, data_ranges=dataset_ranges, w_log=weight_of_log_fits), 
                                data_x, scale_data(data_y, data_in_log, weight_of_log_fits), 
                                [phi0, 412, phi0[2:5]].*2.*rand(10,));

    par_results[:,i1] = fit_orig.param
    par_costs[i1] = sum(scale_data(fit_orig.resid, data_in_log, weight_of_log_fits).^2)
end

y_pred_origfit = model_allmoves(data_x, par_results[:,sortperm(par_costs[:])[1]], data_ranges=dataset_ranges, in_log=data_in_log);



In [737]:
@show minimum(par_costs)
@show par_results[:, sortperm(par_costs[:])[1]]

minimum(par_costs) = 0.08282382732712817
par_results[:,(sortperm(par_costs[:]))[1]] = [20.48349134326988,3.1780011985968315e-6,0.048960008333416107,0.6847457343514155,0.2504910286460144,9.671311734845025,0.0002547793847453547,3.282683170366982,1.094385072300619,0.7437484437258701]


10-element Array{Float64,1}:
 20.4835     
  3.178e-6   
  0.04896    
  0.684746   
  0.250491   
  9.67131    
  0.000254779
  3.28268    
  1.09439    
  0.743748   

In [749]:

p1 = Plot([scatter(;x=data_x[dataset_ranges[3]], y=data_y[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=data_y[dataset_ranges[4]]),
    scatter(;x=data_x[dataset_ranges[3]], y=y_pred_origfit[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=y_pred_origfit[dataset_ranges[4]]),
    ]
        )

p2 = Plot([scatter(;x=data_x[dataset_ranges[1]], y=data_y[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=data_y[dataset_ranges[2]]),
    scatter(;x=data_x[dataset_ranges[1]], y=y_pred_origfit[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=y_pred_origfit[dataset_ranges[2]]),
    ]
        )

plt = plot([p1
    p2])

In [730]:
PlotlyJS.relayout!(plt, height=1000)

In [731]:
#@show par_costs

par_costs = [0.25722266127816085 0.2375544811803338 0.1874841017668799 0.2570124560825571 0.24582472658327023 0.2544240026149982 0.22385165721066913 0.25007818635664003 0.2646593273600224 1.1501888398229454 0.2594756288388401 111.97773923246962 0.10758510968833906 0.26287868371936196 0.2575188238141001 1.5321948039950712 0.24748450912619074 0.2522982770636437 0.2419989046174049 0.25943189597570937 0.2544663646246895 0.21409229043172162 0.26579424921809786 0.28858869751498195 0.26659709195541015 0.10532507047597425 0.21331268250700464 0.27505916797996643 0.8022318406681486 0.24435103786092055 0.27206296320974344 0.24267153350124585 0.24629373236462748 0.2629744235416049 10.18599579436091 0.22689346856013115 0.26644536918484574 0.26021889675424953 0.25190381299188325 0.25806247377805214 0.2620578787802208 0.23806765185874013 0.25692986488025793 0.25162859674955396 0.22346220352584825 0.224229537549055 0.19942449087533198 0.22015835033310938 0.3399068494096438 0.11520711256044282 0.105536

1x100 Array{Float64,2}:
 0.257223  0.237554  0.187484  0.257012  …  0.228809  1.14761  0.105373

In [738]:
# Run MBAM on the complete model and see what happens

basic_model(param::AbstractArray, data::AbstractArray; model_id=0)=model_allmoves(data,param, in_log=data_in_log, data_ranges=dataset_ranges, w_log=weight_of_log_fits);

data_x_grid = [Real(d1) for d1 in data_x];

phi_joint0 = copy(par_results[:, sortperm(par_costs[:])[1]])
#= Code to run in normal julia (so that chrome doesnt die)

outp = MBAM(basic_model, phi_joint0, data_x_grid, model_iters=[0], 
move_dir=[-1], boundary_time=10, verbose=2, reduce_func=BK_reduce);

=#

10-element Array{Float64,1}:
 20.4835     
  3.178e-6   
  0.04896    
  0.684746   
  0.250491   
  9.67131    
  0.000254779
  3.28268    
  1.09439    
  0.743748   